# Adsense Analysis

Recuperamos do Google Analytics dois dataset que contém informações sobre a pesquisa, preço do anuncio que foi disparado, referencia geografica do usuario e estatisticas de conversão. Os datasets estão separados para tentar anonimizar o usuário final, então iremos unir-los e fazer analises em busca de entender o comportamento dos anuncios para uma região

In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
dataset = pd.read_csv('dataset.tsv', sep='\t')
dataset_lat_lng = pd.read_csv('dataset-lat-lng.tsv', sep='\t')

In [3]:
dataset.head()

,adMatchedQuery,date,users,sessions,sessionDuration,adCost,pageviews,pageviewsPerSession,timeOnPage,avgTimeOnPage,exits,pageLoadTime
0,drogasil jardim atlantico,20180412,0,0,0.0,0.30,0,0.0,0.0,0.0,0,0
1,drogasil jardim atlantico,20181009,0,0,0.0,0.30,0,0.0,0.0,0.0,0,0
2,drogasil jardim atlantico,20190129,1,1,28.0,0.30,3,3.0,27.0,13.5,1,0
3,drogasil jardim atlantico,20200129,0,0,0.0,0.24,0,0.0,0.0,0.0,0,0
4,drogasil jardim atlantico,20200222,0,0,0.0,0.27,0,0.0,0.0,0.0,0,0


In [4]:
dataset.shape

(73892, 12)

In [5]:
dataset_lat_lng.head()

,adMatchedQuery,city,latitude,longitude,networkDomain,networkLocation,dateHourMinute,users
0,(not set),Sao Paulo,-23.5505,-46.6333,(not set),(not set),202009141052,1
1,(not set),Sao Paulo,-23.5505,-46.6333,(not set),(not set),202009141053,1
2,(not set),Sao Paulo,-23.5505,-46.6333,(not set),(not set),202010041458,1
3,(not set),Sao Paulo,-23.5505,-46.6333,(not set),(not set),202012070013,1
4,(not set),Sao Paulo,-23.5505,-46.6333,(not set),(not set),202101151632,1


In [6]:
dataset_lat_lng.shape

(52609, 8)

Para unir os datasets usarremos como chaves primárias o termo que foi buscado, o dia que ocorreu e a quantidade de usuarios. Essa ultima chave é para tentarmos ter uma assertividade melhor na localização de um unico usuario ou um grupo.

In [7]:
# Convertendo datas em datetime

dataset.date = pd.to_datetime(dataset.date, format="%Y%m%d").dt.date
dataset_lat_lng['date'] = pd.to_datetime(dataset_lat_lng.dateHourMinute, format="%Y%m%d%H%M").dt.date

In [8]:
# Convertendo o adMatchedQuery para string, limpando acentos, espaços e valores vazios = "(not set)"

def preProcessAdMatchedQuery(dataframe):
    dataframe.adMatchedQuery = dataframe.adMatchedQuery.astype('string')
    dataframe.adMatchedQuery = dataframe.adMatchedQuery.str.lower()
    dataframe.adMatchedQuery = dataframe.adMatchedQuery.str.strip()
    dataframe.adMatchedQuery = dataframe.adMatchedQuery.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    dataframe = dataframe[dataframe.adMatchedQuery != '(not set)']
    return dataframe

dataset = preProcessAdMatchedQuery(dataset)
dataset_lat_lng = preProcessAdMatchedQuery(dataset_lat_lng)

Vamos remover duplicados do dataset nos baseando nas chaves primárias que iremos utilizar para mergir os datasets(adMatchedQuery, date, users)

In [9]:
print('duplicados do dataset', dataset.duplicated(subset=['adMatchedQuery', 'date', 'users']).sum())
print('duplicados do dataset com lat e lng', dataset_lat_lng.duplicated(subset=['adMatchedQuery', 'date', 'users']).sum())
print('shape dataset antes de remover', dataset.shape)
print('shape dataset com lat-lng antes de remover', dataset_lat_lng.shape)
dataset.drop_duplicates(inplace=True, subset=['adMatchedQuery', 'date', 'users'])
dataset_lat_lng.drop_duplicates(inplace=True, subset=['adMatchedQuery', 'date', 'users'])
print('shape dataset depois de remover', dataset.shape)
print('shape dataset lat-lng depois de remover', dataset_lat_lng.shape)

duplicados do dataset 650
duplicados do dataset com lat e lng 18491
shape dataset antes de remover (73118, 12)
shape dataset com lat-lng antes de remover (42437, 9)
shape dataset depois de remover (72468, 12)
shape dataset lat-lng depois de remover (23946, 9)


In [10]:
enriched_dataset = dataset.merge(dataset_lat_lng, on=['adMatchedQuery', 'date', 'users'])

In [11]:
enriched_dataset.shape

(23180, 18)

In [12]:
# Removendo instancia que não são de Recife ou Olinda
enriched_dataset = enriched_dataset[enriched_dataset.city.isin(['Olinda', 'Recife'])]

In [ ]:
# Adicionando o bairro ao dataset usando um geocoder reverso

def getLocality(lat, lng, index):
    print(index)
    request = requests.get(f'https://api.bigdatacloud.net/data/reverse-geocode-client?latitude={lat}&longitude={lng}&localityLanguage=pt')
    return request.json()['locality']

enriched_dataset['locality'] = enriched_dataset.apply(lambda x: getLocality(x.latitude, x.longitude, x.name), axis=1)

In [37]:
enriched_dataset.adCost.value_counts()

0.000000    1570
0.340000    1016
0.280000     996
0.300000     976
0.290000     961
            ... 
0.432599       1
0.280500       1
0.199499       1
0.195600       1
0.112000       1
Name: adCost, Length: 345, dtype: int64